## Importation librairie et donnée


In [1093]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import os
import time
import matplotlib.pyplot as plt

In [1094]:
pd.set_option('display.max_columns', 150)

In [1095]:
def read_and_process_file(fichier_a_deposee):
    vol_prevu = []
    vol_fini = []
    vol_termine = []
    tableau_vol = {}
    tableaux_vol = []
    iter = 0
    flag82 = False
    hneg = False
    compt82 = 0
    num81 = 0
    prevu = False
    termine = False
    final = False
    complet = 0
    output = pd.DataFrame()
    compteur = 0
    tableau_vol = {}
    compteurCcr = 0

    with open(fichier_a_deposee, 'r') as fichier:
        for i, ligne in enumerate(fichier):
            words = ligne.split()
            if words[0] == "02":
                date_str = words[1]
                date_obj = datetime.strptime(date_str, "%d-%m-%Y")
                print('date obj ',date_obj)
                date_fichier = date_obj.timetuple().tm_yday
                print('date fichier ',date_fichier)
            if words[0] == "05":
                if tableau_vol:
                    tableau_vol["isPrevu"] = isprevu
                    tableau_vol["isRealise"] = isrealise
                    tableau_vol["isFinal"] = isfinal
                    df_dictionary = pd.DataFrame([tableau_vol])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
                tableau_vol = {}
                isprevu = False
                isrealise = False
                isfinal = False
                is81 = False
                is82 = False
                balise = ''
                hneg = False
                flag82 = False
                etat = ''
            if words[0] == "11":
                etat = 'prevu'
                isprevu = True
            if words[0] == "12":
                etat = 'realise'
                isrealise = True
            if words[0] == "13":
                if len(words) > 1 and words[1] == "=":
                    for key in list(tableau_vol.keys()):
                        if '_prevu' in key:
                            tableau_vol[key.replace('_prevu', '_final')] = tableau_vol[key]
                etat = 'final'
                isfinal = True
            if words[0] == "14":
                etat = 'transaction'
            if words[0] == "81":
                pass
            if words[0] == "20":
                tableau_vol['callSign_' + etat] = words[1]
                tableau_vol['dep_' + etat] = words[2]
                tableau_vol['arr_' + etat] = words[3]
                tableau_vol['numCautra_' + etat] = words[4]
                tableau_vol['dateRelative_' + etat] = words[5]
                tableau_vol['typeAvion_' + etat] = words[6]
                tableau_vol['work_' + etat] = words[7]
                if words[8][:2] == '??':
                    pass
                else:
                    tableau_vol['work1' + etat] = words[8].strip().ljust(9)
            if words[0] == "21":
                tableau_vol['heuresDep_' + etat] = words[1]
                tableau_vol['RFL_' + etat] = words[2]
                tableau_vol['vitesse_' + etat] = words[3]
                tableau_vol['EOBT_' + etat] = words[4]
            if words[0] == "22":
                tableau_vol['regleVol_' + etat] = words[1]
                tableau_vol['typeVol_' + etat] = words[2]
                tableau_vol['HeurePremiereBaliseActive_' + etat] = words[10]
                if words[3][:2] == '??':
                    pass
                else:
                    tableau_vol['IFPL_' + etat] = words[3].strip().ljust(10)
                tableau_vol['plnActive_' + etat] = words[4]
                tableau_vol['typePlnStan']= words[6]
                tableau_vol['plnAnnule_' + etat] = words[5]
                if '??' in words[7]:
                    pass
                elif len(words[7]) == 8:
                    date_str = words[7]
                    date_obj = datetime.strptime(date_str, '%d%m%Y')
                    day_vol = date_obj.timetuple().tm_yday
                    tableau_vol['dateBlock_' + etat] = words[7][:4] + words[7][6:]
                else:
                    tableau_vol['dateBlock_' + etat] = words[7].strip().ljust(6)
            if words[0] == "23":
                if "??" in words[4]:
                    tableau_vol['adresseModeS_' + etat] = np.NaN
                else:
                    tableau_vol['adresseModeS_' + etat] = words[4]
            if words[0] == "24":
                tableau_vol['numeroPLNM' + etat] = words[1]
                tableau_vol['flightID' + etat] = words[2]
            if words[0] == "31":
                tableau_vol['balise' + etat] = words[1]
            if words[0] == "32":
                tableau_vol['HeurePremiereBalise_' + etat] = words[1]
            if words[0] == "33":
                tableau_vol['listeBalises' + etat] = words[1]
            if words[0] == "36":
                tableau_vol['indicateur' + etat] = words[1]
            if words[0] == "41":
                tableau_vol['carte' + etat] = words[1]
            if words[0] == "71":
                tableau_vol['centreTraversé' + etat] = words[1:]
            if words[0] == "72":
                tableau_vol['listeRangPremier' + etat] = words[1]
            if words[0] == "80":
                tableau_vol['rangTransaction' + etat] = words[1]
            if words[0] == "81" and not is81:
                is81 = True
                if len(words) >= 15:
                    parts = ligne.split("-")
                    last_word = parts[0].split()[-1]
                    if "ABI" in ligne:
                        tableau_vol['typePln'] = "ABI"
                    if "(FPL" in parts[0] or "(CHG)" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[8]
                            if tableau_vol['case18'] == "RPL":
                                tableau_vol['typePln'] = "RPL"
                        else:
                            #print(ligne)
                            compteur += 1
                    elif "(APL" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        tableau_vol['typePln'] = "APL"
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[-1]
                        else:
                            print(ligne)
                            compteur += 1
            if words[0] == "82"and not is82:
                is82 = True
                tableau_vol['heure'] = (words[1][:2])
                tableau_vol['minute'] = (words[1][3:])
                tableau_vol['accuseTrt' + etat] = words[1]
                if "CCR:" in ligne:
                    compteurCcr = 0
                    for word in words:
                        compteurCcr += 1
                        if word == "CCR:":
                            break
                    tableau_vol['ccrArrival'] = words[compteurCcr]
            if words[0] == "84":
                tableau_vol['final' + etat] = words[1]

    return output

def convert_and_calculate(df,date_obj):
    df['HeurePremiereBaliseActive_realise'] = df['HeurePremiereBaliseActive_realise'].astype('Int64')
    df['HeurePremiereBaliseActive_final'] = df['HeurePremiereBaliseActive_final'].astype('Int64')
    df['HeurePremiereBalise_final'] = df['HeurePremiereBalise_final'].astype('Int64')
    df['dateRelative_realise'] = df['dateRelative_realise'].astype('Int64')
    df['dateRelative_final'] = df['dateRelative_final'].astype('Int64')

    def calcul_HeureDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if not pd.isna(row['HeurePremiereBaliseActive_realise']) and int(row['HeurePremiereBaliseActive_realise']) != 0:
                    return int(row['HeurePremiereBaliseActive_realise']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
            elif not pd.isna(row['dateRelative_final']):
                if not pd.isna(row['HeurePremiereBaliseActive_final']) and int(row['HeurePremiereBaliseActive_final']) != 0:
                    return int(row['HeurePremiereBaliseActive_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
        except Exception:
            return None

    df['heure_de_reference'] = df.apply(calcul_HeureDeReference, axis=1)

    def calcul_DateDeReference(row):
        print('date obj func date ref',date_obj)

        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if row['dateRelative_realise'] == 0:
                    return date_obj
                elif row['dateRelative_realise'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_realise'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
            elif not pd.isna(row['dateRelative_final']) and not pd.isnull(row['dateRelative_final']):
                if row['dateRelative_final'] == 0:
                    return date_obj
                elif row['dateRelative_final'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_final'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
        except Exception as e:
            return None

    df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)
    return df

In [1096]:
from datetime import datetime, timedelta

def dynamic_analysis(target_date_str):
    # Convert the target_date_str to a datetime object
    target_date = datetime.strptime(target_date_str, "%d-%m-%Y")
    print('target date',target_date)
    # Calculate the previous and next days
    previous_day = target_date - timedelta(days=1)
    print('previous_day',previous_day)
    next_day = target_date + timedelta(days=1)
    print('next_day',next_day)
    
    # Format the dates to match the file naming convention
    previous_day_str = previous_day.strftime("%Y%m%d")
    target_date_str = target_date.strftime("%Y%m%d")
    next_day_str = next_day.strftime("%Y%m%d")
    
    # Construct file names
    nom_fichier_jour_1 = f'../data/stan_mars_2024/RDVC-{previous_day_str}.pln'
    nom_fichier_jour_2 = f'../data/stan_mars_2024/RDVC-{target_date_str}.pln'
    nom_fichier_jour_3 = f'../data/stan_mars_2024/RDVC-{next_day_str}.pln'
    print('jour de veille',nom_fichier_jour_1)
    print('jour de jour',nom_fichier_jour_2)
    print('jour de lendemain',nom_fichier_jour_3)
    
    # Read and process files
    fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
    fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
    fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)
    
    # Convert and calculate
    date_obj = previous_day
    jour_1 = convert_and_calculate(fichier_jour_1,date_obj)
    print('date obj ',date_obj)

    date_obj = target_date
    jour_2 = convert_and_calculate(fichier_jour_2,date_obj)
    print('date obj ',date_obj)

    date_obj = next_day
    jour_3 = convert_and_calculate(fichier_jour_3,date_obj)
    print('date obj ',date_obj)
    
    # Return the results
    return jour_1, jour_2, jour_3

# Example usage
dateAnalyse='24-03-2024'
jour_1, jour_2, jour_3 = dynamic_analysis(dateAnalyse)

target date 2024-03-24 00:00:00
previous_day 2024-03-23 00:00:00
next_day 2024-03-25 00:00:00
jour de veille ../data/stan_mars_2024/RDVC-20240323.pln
jour de jour ../data/stan_mars_2024/RDVC-20240324.pln
jour de lendemain ../data/stan_mars_2024/RDVC-20240325.pln
date obj  2024-03-23 00:00:00
date fichier  83
date obj  2024-03-24 00:00:00
date fichier  84
date obj  2024-03-25 00:00:00
date fichier  85
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:00:00
date obj func date ref 2024-03-23 00:

In [1097]:
# nom_fichier_jour_1 = '../data/stan_mars_2024/RDVC-20240315.pln'
# nom_fichier_jour_2 = '../data/stan_mars_2024/RDVC-20240316.pln'
# nom_fichier_jour_3 = '../data/stan_mars_2024/RDVC-20240317.pln'
# fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
# fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
# fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)

In [1098]:
# date_obj = datetime.strptime('15-03-2024', "%d-%m-%Y")
# jour_1 = convert_and_calculate(fichier_jour_1)
# date_obj = datetime.strptime('16-03-2024', "%d-%m-%Y")
# jour_2 = convert_and_calculate(fichier_jour_2)
# date_obj = datetime.strptime('17-03-2024', "%d-%m-%Y")
# jour_3 = convert_and_calculate(fichier_jour_3)

In [1099]:
jour_1['date_de_reference'].value_counts()

date_de_reference
2024-03-23    7956
2024-03-22     227
Name: count, dtype: int64

# Algo 2.3 et 2.4

## Tables utilisées

In [1100]:
INDICATIF_FICTIF = pd.read_csv("../data/TABLE_SYSTEME/indicatifsfictifs", sep=';', skiprows=3)
EUROPE_SUD = pd.read_csv("../data/TABLE_SYSTEME/payssud", sep=';', skiprows=3)
AERONEFS_DE_MOINS_DE_2_TONNES = pd.read_csv("../data/TABLE_SYSTEME/aeronefsmoin2tonnes", sep=';', skiprows=3)
INDICATEURS_D_EMPLACEMENT_FAUX = pd.read_csv("../data/TABLE_SYSTEME/aerodromesfaux", sep=';', skiprows=3)
AERODROME_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/aerodromesaverifier", sep=';', skiprows=3)
AERODROMES_D_APPROCHE = pd.read_csv("../data/TABLE_SYSTEME/aerodromesapproche", sep=';', skiprows=3)
AERODROMES_FRONTALIERS = pd.read_csv("../data/TABLE_SYSTEME/aerodromesfrontaliers", sep=';', skiprows=3)
INDICATIF_A_CODE_AUTO = pd.read_csv("../data/TABLE_SYSTEME/indicatifscodesauto", sep=';', skiprows=3)
INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE = pd.read_csv("../data/TABLE_SYSTEME/structurtrigrammecorrects", sep=';', skiprows=3)
COMPAGNIES_BIGRAMMES_ET_SUFFIXES = pd.read_csv("../data/TABLE_SYSTEME/cgniebigrammesetsuffixes", sep=';', skiprows=3)
COMPAGNIES_AVEC_TRIGRAMME = pd.read_csv("../data/TABLE_SYSTEME/compagniesavectrigrammesautorise", sep=';', skiprows=3)
INDICATIFS_TRIGRAMME_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/structuretrigrammeverif", sep=';', skiprows=3)
LETTRES_AIR_FRANCE = pd.read_csv("../data/TABLE_SYSTEME/lettresairfrance", sep=';', skiprows=3)
LETTRES_AIR_FRANCE = LETTRES_AIR_FRANCE[~((LETTRES_AIR_FRANCE['Lettre'].str.contains("MODIFIE", na=False)) & (LETTRES_AIR_FRANCE['Code exo/trait'].isna()) & (LETTRES_AIR_FRANCE['Commentaire'].isna()))]
LETTRES_AIR_FRANCE.reset_index(drop=True, inplace=True)
IMMATRICULATION_A_VERIFIER = pd.read_csv("../data/TABLE_SYSTEME/immataverif", sep=';', skiprows=3)
IMMATRICULATION_CORRECTE = pd.read_csv("../data/TABLE_SYSTEME/immatcorrectes", sep=';', skiprows=3)
OPERATEURS_MILITAIRES = pd.read_csv("../data/TABLE_SYSTEME/oprmili", sep=';', skiprows=3)
AERONEFS_STRICTEMENT_MILITAIRES = pd.read_csv("../data/TABLE_SYSTEME/aeronefmilitaire", sep=';', skiprows=3)

In [1101]:
# Fonction pour convertir un modèle en expression régulière
def pattern_to_regex(pattern):
    # Remplacer # par .*, + par [a-zA-Z] et - par [0-9]
    regex = pattern.replace('#', '.?').replace('-', '[0-9]').replace('+', '[a-zA-Z]')
    # Ajouter les délimiteurs de début et de fin
    regex = '^' + regex + '$'
    return regex

def trouver_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return True
    return False

def renvoie_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return pattern
    return "NULL"

## Prétraitement utile/inutile

In [1102]:
def nbre_centre_francais(list):
    rep = 0
    for x in list:
      if x in ['PARI', 'AIX', 'BRST', 'REIM', 'BORD']:
        rep +=1
    return rep

In [1103]:
def utile_inutile(element):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return None

    call_sign_value = get_valid_value(element, ["callSign_realise", "callSign_final", "callSign_prevu"])
    PLN_active_value = get_valid_value(element, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
    PLN_annule_value = get_valid_value(element, ["plnAnnule_realise", "plnAnnule_final", "plnAnnule_prevu"])
    centre_traverse_value = get_valid_value(element, ["centreTraversérealise", "centreTraverséfinal", "centreTraverséprevu"])
    dep_value = get_valid_value(element, ["dep_realise", "dep_final", "dep_prevu"])
    arr_value = get_valid_value(element, ["arr_realise", "arr_final", "arr_prevu"])

    if trouver_pattern(call_sign_value, INDICATIF_FICTIF["Indicatif"].to_list()):
        return "FICT"
    if PLN_active_value == "1":
        return "UTI"
    elif PLN_annule_value == "1":
        return "CNL"
    elif nbre_centre_francais(centre_traverse_value) >= 2:
        return "2SLF"
    elif dep_value == arr_value:
        return "CIRC"
    elif dep_value[0] == "E" and dep_value in EUROPE_SUD["Code pays"].to_list():
        return "NORD"
    elif (dep_value[0] in EUROPE_SUD["Code pays"].to_list() or dep_value[:2] in EUROPE_SUD) and dep_value[0] == "E":
        return "SUD"
    return "UTI"


In [1104]:
def traitement_utile_inutile(df):
    df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)
    return df

In [1105]:
def test_utile_vol_active_non_fictif():
    utilite = result[result["callSign_prevu"]=="TRA79Y"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_inutile_vol_fictif_avec_moins():
    utilite = result[result["callSign_prevu"]=="RFIN65"]["utile_inutile"].iloc[0]
    assert utilite == "FICT", "utilité diff de FICT"

def test_inutile_vol_annule():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_inutile_vol_centrefr():
    utilite = result[result["callSign_prevu"]=="TRA4N"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

def test_inutile_vol_circulaire():
    utilite = result[result["callSign_prevu"]=="FNY5118"]["utile_inutile"].iloc[0]
    assert utilite == "CIRC", "utilité diff de CIRC"

def test_utile_vol_tout_les_test_faux():
    utilite = result[result["callSign_prevu"]=="TOM22B"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_realise_pln_nan():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_realise_centre_nan():
    utilite = result[result["callSign_prevu"]=="UAL71"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

## Traitement unitaire

## Initialisation

In [1106]:
def TU_init(df_utile):
    df_utile['aeronef_de_moins_de_deux_tonnes'] = False
    df_utile['vol_a_transmettre'] = False
    df_utile['vol_approche'] = False
    df_utile['vol_interieur'] = False
    df_utile['vol_frontalier'] = False
    df_utile['vol_VFR'] = False
    df_utile["code_d_exoneration"] = 'Z'
    df_utile['code_exploitant'] = 'Z'
    df_utile['compagnie_française'] = False
    df_utile['type_d_avion_militaire'] = False
    df_utile['PLN_à_verifier_TU'] = False
    df_utile['PLN_valide'] = True
    df_utile['RAZ_des_invalidites_de_type_TU'] = False
    df_utile['invalidite_TU'] = [[] for _ in range(len(df_utile))]
    df_utile['type_d_indicatif'] = ""
    return df_utile

## Algo 1

In [1107]:
def TU_1(df_utile):
    def TU_1_element(x):
        # Vérification de PLN_active
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "0":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["NACT"]
                break

        # Vérification de typeavion
        for col in ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "ZZZZ":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["TYPAV"]
                    x['PLN_valide'] = False
                if x[col] in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
                    x['aeronef_de_moins_de_deux_tonnes'] = True
                break
        return x
    df_utile = df_utile.apply(TU_1_element, axis=1)
    return df_utile

## Algo 2

In [1108]:
# def TU_2(df_utile):
#     def TU_2_element(x):
#         if len(x['depfinal']) != 4:
#             x['invalidite_TU'].append("DEPAR1")
#             x['PLN_valide'] = False
#         elif x['depfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("DEPAR2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['depfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("DVPAR3")
#             x['PLN_valide'] = False
#         if len(x['arrfinal']) != 4:
#             x['invalidite_TU'].append("ARRIV1")
#             x['PLN_valide'] = False
#         elif x['arrfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("ARRIV2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['arrfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("AVRIV3")
#             x['PLN_valide'] = False
#     df_utile = df_utile.apply(TU_2_element, axis=1)
#     return df_utile

In [1109]:
def TU_2(df_utile):
    def TU_2_element(x):
        # Vérification des colonnes de départ
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("DEPAR1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("DEPAR2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("DVPAR3")
                    x['PLN_valide'] = False
            break

        # Vérification des colonnes d'arrivée
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("ARRIV1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("ARRIV2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("AVRIV3")
                    x['PLN_valide'] = False
            break
        return x
    df_utile = df_utile.apply(TU_2_element, axis=1)
    return df_utile

## Algo 3

In [1110]:
def TU_3_bis(df_utile):
    eurocontrol = ['EB','ED','EF','EG','EH','EI','EK','EL','EN','EP','ES','EV','EY','LA','LB','LC','LD','LE','LG','LH','LI','LJ','LK','LM','LO','LP','LQ','LR','LS','LT','LU','LW','LY','LZ','UD','UG','GC']
    def TU_3_bis_element(x):
        dep_value = None
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        if dep_value == "ZZZZ":
            x['PLN_valide'] = False
            x['PLN_à_verifier_TU'] = False
            x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
        elif dep_value[:2] in eurocontrol:
            x['vol_a_transmettre'] = False
        elif x["ccrArrival"] == "ALGR":
            x['vol_a_transmettre'] = True
        elif x["ccrArrival"] == "ETRG":
            if ((x['dep_prevu'][:1]=='D' or x['dep_prevu'][:1]=='F') and 'TABOT' in x['case15']):
                x['vol_a_transmettre'] = True
        else:
            x['vol_a_transmettre'] = False
        return x

    df_utile = df_utile.apply(TU_3_bis_element, axis=1)
    return df_utile

In [1111]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         if x["depfinal"] == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].append("DEPAR2")
#             x['invalidite_TU'].append("TRANS1")
#         else:
#             if x["depfinal"][:2] in liste_origin_vol | x["depfinal"][:2] in liste_origin_vol:
#                 if trouver_code(x["depfinal"], liste_origin_vol) == "A":
#                     x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "B":
#                     x['vol_a_transmettre'] = True
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "C":
#                     if x["AVR"] == 1:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSC")
#                     else:
#                         x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "D":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSD")
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "E":
#                     if x["SVR"] == 1:
#                         x['vol_a_transmettre'] = False
#                     else:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSE")
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSI")
#             else:
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = True
#                 else:
#                     if x["balisefinal"] == "PTGEO":
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSP")
#                     else:
#                         x['vol_a_transmettre'] = False
#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

In [1112]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         dep_value = None
#         balise_value = None

#         # Vérification des colonnes de départ
#         for col in ["deprealise", "depfinal", "depprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 dep_value = x[col]
#                 break
#         for col in ["baliserealise", "balisefinal", "baliseprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 balise_value = x[col]
#                 break

#         if dep_value == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
#         elif dep_value:
#             dep_code = trouver_code(dep_value, liste_origin_vol)
#             if dep_code == "A":
#                 x['vol_a_transmettre'] = False
#             elif dep_code == "B":
#                 x['vol_a_transmettre'] = True
#             elif dep_code == "C":
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSC")
#                 else:
#                     x['vol_a_transmettre'] = False
#             elif dep_code == "D":
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSD")
#             elif dep_code == "E":
#                 if x["SVR"] == 1:
#                     x['vol_a_transmettre'] = False
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSE")
#             else:
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSI")
#         else:
#             if x["AVR"] == 1:
#                 x['vol_a_transmettre'] = True
#             else:
#                 if balise_value == "PTGEO":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSP")
#                 else:
#                     x['vol_a_transmettre'] = False

#         return x

#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

##  Algo 4

In [1113]:
def TU_4(df_utile):
    def TU_4_element(x):
        x['vol_a_transmettre'] = True
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                arr_value = x[col]
                break
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                plnActive = x[col]
                break

        if dep_value in AERODROMES_D_APPROCHE["Code terrain"].to_list():
            x["vol_approche"] = True

        if arr_value[:2] == "LF":
            x["vol_interieur"] = True

        elif  plnActive == 0 and dep_value in AERODROMES_FRONTALIERS["Code terrain"].to_list():
            x["vol_frontalier"] = True

        return x
    df_utile = df_utile.apply(TU_4_element, axis=1)
    return df_utile

In [1114]:
# def TU_4(df_utile):
#     def TU_4_element(x):
#         x['vol_a_transmettre'] = True
#         if x["depfinal"] in AERODROMES_D_APPROCHE:
#             x["vol_approche"] = True
#         if x["arrfinal"][:2] == "LF":
#             x["vol_interieur"] == True
#         elif x["PLN_activefinal"] != 1 & x["depfinal"] in AERODROMES_FRONTALIERS:
#             x["vol_frontalier"] == True
#     df_utile = df_utile.apply(TU_4_element, axis=1)
#     return df_utile

## Algo 5

In [1115]:
def TU_5(df_utile):
    def TU_5_element(x):
        # Vérification des colonnes balise et regleVol
        balise_value = None
        regleVol_value = None
        dep_value = None
        for col in ["baliserealise", "balisefinal", "baliseprevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                balise_value = x[col]
                break
        for col in ["regleVol_realise", "regleVol_final", "regleVol_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                regleVol_value = x[col]
                break
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        # Conditions basées sur les valeurs trouvées
        if x["typePln"] == "VFR" and balise_value.startswith('VFR') and regleVol_value == "V":
            x['vol_a_transmettre'] = False
        elif x["typePln"] in ["APL", "FPL", "FIH", "FII"] and balise_value.startswith('VFR'):
            x['vol_a_transmettre'] = False
            x['vol_VFR'] = True
            x["typePln"] = "VFR"
        else:
            if not x.get('vol_VFR', False) and regleVol_value == "V" and x.get('vol_a_transmettre', True):
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS0")
            elif dep_value and dep_value[:2] != "LF" and not x.get('vol_a_transmettre', True) and regleVol_value == "Z":
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS2")
                x["typePln"] = "AFI"
            if not x.get('vol_VFR', False) and any(s.startswith('TRANS') for s in x['invalidite_TU']):
                x['vol_a_transmettre'] = "ADET"
        if x["typePln"] in ["APL", "FPL"] and regleVol_value in ["Y", "Z"] and x.get('vol_a_transmettre', True):
            x["code_d_exoneration"] = "Y"
        return x
    df_utile = df_utile.apply(TU_5_element, axis=1)
    return df_utile


In [1116]:
# def TU_5(df_utile):
#     def TU_5_element(x):
#         if x["typePln"] == "VFR" and x["baliseprevu"].str.startswith('VFR') and x["regleVol_prevu"] == "V":
#             x['vol_a_transmettre'] = False
#         elif (x["typePln"] == "APL" or x["typePln"] == "FPL" or x["typePln"] == "FIH" or x["typePln"] == "FII") and x["baliseprevu"].str.startswith('VFR'):
#             x['vol_a_transmettre'] = False
#             x['vol_VFR'] = True
#             x["typePln"] = "VFR"
#         else:
#             if x['vol_VFR'] == False and x["regleVol_prevu"] == "V" and x['vol_a_transmettre'] == True:
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS0")
#             elif x["depprevu"][:2] != "LF" and x['vol_a_transmettre'] == False and x["regleVol_prevu"] == "Z":
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS2")
#                 x["typePln"] = "AFI"
#             if x['vol_VFR'] == False and any(s.startswith('TRANS') for s in x['invalidite_TU']):
#                 x['vol_a_transmettre'] = "ADET"
#         if (x["typePln"] == "APL" or x["typePln"] == "FPL") and ( x["regleVol_prevu"] == "Y" or  x["regleVol_prevu"] == "Z") and x['vol_a_transmettre'] == True:
#             x["code_d_exoneration"] = "Y"
#     df_utile = df_utile.apply(TU_5_element, axis=1)
#     return df_utile

## Algo 6

In [1117]:
def TU_65_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
        x['type_d_avion_militaire'] = True
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["TYPA19", "INDI19"])
        filtre_avion = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
        if not pd.isna(filtre_avion["Code exoneration"].iloc[0]):
            x["code_d_exoneration"] = filtre_avion["Code exoneration"].iloc[0]
            x['code_exploitant'] = filtre_avion["Code exploitant"].iloc[0]
        else:
            x['code_exploitant'] = 'Z'
    return x

In [1118]:
def TU_64_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    OPR = None
    if not(pd.isna(x["case18"])):
        OPR = trouver_case18("OPR/", x["case18"])
    if not(pd.isna(x["case18"])) and OPR != "NULL" and OPR in OPERATEURS_MILITAIRES['Nom operateur'].tolist():
        operateur_militaire = OPERATEURS_MILITAIRES[OPERATEURS_MILITAIRES["Nom operateur"] == OPR]
        x["code_d_exoneration"] = operateur_militaire["Code exoneration"].iloc[0]
        x['code_exploitant'] = operateur_militaire["Code exploitant"].iloc[0]
    else:
        if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
            x['type_d_avion_militaire'] = True
            avion_militaire = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
            if not pd.isna(avion_militaire["Code exoneration"].iloc[0]):
                x["code_d_exoneration"] = avion_militaire["Code exoneration"].iloc[0]
                x['code_exploitant'] = avion_militaire["Code exploitant"].iloc[0]
            else:
                x['code_exploitant'] = 'Z'
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["EXO19", "INDI19", "OPR19"])
    return x

In [1119]:
def TU_63_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and trouver_pattern(call_sign_value, IMMATRICULATION_CORRECTE["Immatriculation"]):
        if trouver_pattern(call_sign_value, IMMATRICULATION_A_VERIFIER["Immatriculation"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC$", "EXO$", "OPR$"])
        else:
            x['type_d_indicatif'] = "IM"
        x = TU_65_element(x)
    else:
        x = TU_64_element(x)
    return x

In [1120]:
def TU_62_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"].values:
        x['code_exploitant'] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exploitant"].iloc[0]
        if x['vol_interieur'] == True:
            x["code_d_exoneration"] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exoneration"].iloc[0]
            x['compagnie_française'] = True
            x['type_d_indicatif'] = "BI"
        else:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC%", "EXO%", "OPR%"])
        x = TU_65_element(x)
    return x

In [1121]:
def TU_612_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None

    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])

    if call_sign_value and re.search(r'\d[A-Z]$', call_sign_value) and call_sign_value[-1] == -1:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("INDIC6")
    x = TU_65_element(x)
    return x

In [1122]:
def TU_611_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[-1] in LETTRES_AIR_FRANCE['Lettre'].tolist():
        code_air_france = LETTRES_AIR_FRANCE[LETTRES_AIR_FRANCE['Lettre'] == call_sign_value[-1]]["Code exo/trait"].iloc[0]
        if code_air_france == 2:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC8", "EXO8"])
        elif not code_air_france.isdigit():
            x["code_d_exoneration"] = code_air_france
    else:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("EXO10")
    x = TU_65_element(x)
    return x

In [1123]:
def TU_61_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value[:3] in COMPAGNIES_AVEC_TRIGRAMME["Trigramme"].tolist():
        x['type_d_indicatif'] = "TR"
        if trouver_pattern(call_sign_value, INDICATIFS_TRIGRAMME_A_VERIFIER["Indicatif"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC5", "EXO5"])
        elif call_sign_value[:3] == "AFR":
            x['compagnie_française'] = True
            if re.search(r'\d[A-Z]$', call_sign_value):
                x = TU_611_element(x)
            else:
                x = TU_65_element(x)
        else:
            x = TU_612_element(x)
    else:
        x = TU_64_element(x)
    return x

In [1124]:
def TU_6(df_utile):
    def TU_6_element(x):
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not pd.isna(element[col]) and element[col] != "":
                    return element[col]
            return None
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        if call_sign_value and trouver_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist()):
            Indicatif = renvoie_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist())
            x["code_d_exoneration"] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exoneration"].iloc[0]
            x['code_exploitant'] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exploitant"].iloc[0]
            if x["code_d_exoneration"] in ["X", "M"]:
                x['type_d_avion_militaire'] = True
        else:
            if trouver_pattern(call_sign_value, INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE["Indicatif"].tolist()):
                x = TU_61_element(x)
            elif re.match(r'^[A-Z]{2}\d{3}[A-Z]{2}$', call_sign_value) and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Bigramme"].tolist():
                x = TU_62_element(x)
            else:
                x = TU_63_element(x)
        return x
    df_utile = df_utile.apply(TU_6_element, axis=1)
    return df_utile

## Algo 7

In [1125]:
# Chaîne d'entrée
def trouver_case18(prefix, input_string):
    index_find = input_string.find(prefix)
    if index_find == -1:
        return "NULL"
    else:
        start_index = index_find + len(prefix)
        end_index = input_string.find(' ', start_index)
        opr_code = input_string[start_index:end_index]
        return opr_code

In [1126]:
# def TU_7(df_utile):
#     def TU_7_element(x):
#         if x["PLN_activeprevu"] == "0" and x['vol_a_transmettre'] == True:
#             x['PLN_valide'] = False
#         if x["callSign_prevu"] == "APL" or x["callSign_prevu"] == "FPL":
#             if x['vol_a_transmettre'] == True:
#                 if x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z":
#                     if x['typeVol_prevu'] == "M":
#                         x['PLN_valide'] = False
#                         x['invalidite_TU'].append("EXO10")
#                         x['invalidite_TU'].append("OPR10")
#                     else:
#                         if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True:
#                             x["code_d_exoneration"] = "T"
#                         elif x['typeVol_prevu'] == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
#                             x['PLN_valide'] = trouver_case18
#                             x['invalidite_TU'].append("EXO12")
#             else:
#                 if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True and (x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z"):
#                     x["code_d_exoneration"] = "T"

#     df_utile = df_utile.apply(TU_7_element, axis=1)
#     return df_utile

In [1127]:
def TU_7(df_utile):
    def TU_7_element(x):
        # Fonction pour trouver une valeur valide dans les colonnes multiples
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not(element[[col]].isna().iloc[0]):
                    return element[col]
            return None

        PLN_active_value = get_valid_value(x, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        type_vol_value = get_valid_value(x, ["typeVol_realise", "typeVol_final", "typeVol_prevu"])

        if PLN_active_value == "0" and x.get('vol_a_transmettre', False):
            x['PLN_valide'] = False

        if call_sign_value in ["APL", "FPL"]:
            if x.get('vol_a_transmettre', False):
                if code_exoneration_value in ["Y", "Z"]:
                    if type_vol_value == "M":
                        x['PLN_valide'] = False
                        x['invalidite_TU'].extend(["EXO10", "OPR10"])
                    else:
                        if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False):
                            x["code_d_exoneration"] = "T"
                        elif type_vol_value == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
                            x['PLN_valide'] = False
                            x['invalidite_TU'].append("EXO12")
            else:
                if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False) and code_exoneration_value in ["Y", "Z"]:
                    x["code_d_exoneration"] = "T"
        return x
    df_utile = df_utile.apply(TU_7_element, axis=1)
    return df_utile

## Traitement unitaire complet

In [1128]:
def traitement_unitaire(df_utile):
    df_utile = TU_init(df_utile)
    df_utile = TU_1(df_utile)
    df_utile = TU_2(df_utile)

    def check_depfinal_prefix(row):
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in row and not pd.isna(row[col]) and row[col] != "":
                if row[col][:2] == "LF":
                    return True
        return False

    def apply_transformations(row):
        if check_depfinal_prefix(row):
            return TU_4(pd.DataFrame([row])).iloc[0]
        else:
            return TU_3_bis(pd.DataFrame([row])).iloc[0]

    # Appliquer les transformations par ligne
    df_utile = df_utile.apply(apply_transformations, axis=1)

    df_utile = TU_5(df_utile)
    df_utile = TU_6(df_utile)
    df_utile = TU_7(df_utile)
    return df_utile

# Transformation des données RDVC

In [1129]:
jour_1_utile_inutile = traitement_utile_inutile(jour_1)
jour_2_utile_inutile = traitement_utile_inutile(jour_2)
jour_3_utile_inutile = traitement_utile_inutile(jour_3)

In [1130]:
jour_1_utile = jour_1_utile_inutile[jour_1_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_2_utile = jour_2_utile_inutile[jour_2_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_3_utile = jour_3_utile_inutile[jour_3_utile_inutile["utile_inutile"] == "UTI"].copy()

In [1131]:
jour_1_utile_traitee = traitement_unitaire(jour_1_utile)
jour_2_utile_traitee = traitement_unitaire(jour_2_utile)
jour_3_utile_traitee = traitement_unitaire(jour_3_utile)

Fonction
ajouter pour chaque df une colonne doublon à False
Verifier que si un call sign et une heure de reference à 19mn pret (à parametrer) indiquer doublon sur l'un des vols seulement

In [1132]:
seuil_doublon = 19

In [1133]:
import pandas as pd

def ajouter_colonne_doublon(df, tolerance_minutes=19):

    df['doublon'] = False
    

    df = df.sort_values(by=['callSign_prevu', 'heure_de_reference']).reset_index(drop=True)

    i = 1
    while i < len(df):
        if df.at[i, 'callSign_prevu'] == df.at[i - 1, 'callSign_prevu']:
            delta = abs((df.at[i, 'heure_de_reference'] - df.at[i - 1, 'heure_de_reference']))
            print()
            print('delta = ', delta)
            if delta <= tolerance_minutes:
                df = df.drop(i).reset_index(drop=True)
                continue  # Réévaluer le même index i après la suppression et la réinitialisation de l'index
        i += 1

    return df

def supprimer_doublons(df):
    return
    
    return df[df['doublon'] == False].drop(columns=['doublon'])

In [1134]:
jour_1_utile_traitee.head()

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,work1prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,work1final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,numeroPLNMfinal,flightIDfinal,balisefinal,HeurePremiereBalise_final,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,work1realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,case7,case8,case9,case10,case13,case15,case16,case18,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,typePln,heure_de_reference,date_de_reference,utile_inutile,aeronef_de_moins_de_deux_tonnes,vol_a_transmettre,vol_approche,vol_interieur,vol_frontalier,vol_VFR,code_d_exoneration,code_exploitant,compagnie_française,type_d_avion_militaire,PLN_à_verifier_TU,PLN_valide,RAZ_des_invalidites_de_type_TU,invalidite_TU,type_d_indicatif
0,TOM4CT,HEGN,EGBB,7374,-1,B38M,0000,GTUMX,-480,340,453,-480,I,S,0,AA57120257,0,FPL,0,220324,NaN,-1,????????,DITON,-215,380,0,HH,"[ZURI, REIM, EGGG]",1,TOM4CT,HEGN,EGBB,7374,-1,B38M,2744,GTUMX,-480,340,453,-480,I,S,0,AA57120257,0,0,220324,NaN,-1,????????,DITON,-219,380,0,6M,"[ZURI, REIM, EGGG]",1,TOM4CT,HEGN,EGBB,-1,-1,B38M,0000,GTUMX,-480,380,453,-480,I,S,1228,AA57120257,1,0,220324,NaN,-1,????????,PTGEO,-212,380,0,HH,[REIM],1,2,TOM4CT,IS,B38M/M,SDFGHIJ1RWXYZ/LB1,HEGN1600,N0453F340 SOLOX1B SOLOX V603 GIDID DCT KAPIT L...,EGBB0539 EGNX,CODE/407E7B DOF/240322 EE\n,17,23,17:23,ZURI,TOM4CT,True,True,True,NaN,-212,2024-03-22,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR
1,MSR996,CYYZ,HECA,7961,-1,B77W,0000,SUGDO,-435,330,473,-435,I,S,0,AA57132833,0,FPL,0,220324,NaN,-1,????????,LULOX,-118,350,0,S9,"[EGGG, BRST, REIM, ZURI]",1,MSR996,CYYZ,HECA,7961,-1,B77W,1144,SUGDO,-435,330,473,-435,I,S,0,AA57132833,1,0,220324,NaN,-1,????????,LULOX,-86,350,0,S9,"[EGGG, BRST, REIM, ZURI]",1,MSR996,CYYZ,HECA,7961,-1,B77W,1144,SUGDO,-435,370,473,-435,I,S,1368,AA57132833,1,0,220324,01013D,-1,MSR996,LULOX,-72,350,0,S9,"[EGGG, BRST, REIM, ZURI]",1,4,MSR996,IS,B77W/H,SDE1E2E3FGHIJ2J3J4J5M1P2RWXYZ/LB1D1,CYYZ1645,N0473F330 VERDO7 LORKA Q907 ADVIK DCT RABIK DC...,NaN,NaN,19,19,19:19,EGGG,MSR996,True,True,True,ABI,-72,2024-03-22,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR
2,TOM44X,HEGN,EGGD,7784,-1,B38M,0000,GTUMM,-420,340,452,-420,I,S,0,AA57121294,0,FPL,0,220324,NaN,-1,????????,DITON,-155,380,0,HH,"[ZURI, REIM, EGGG]",1,TOM44X,HEGN,EGGD,7784,-1,B38M,7525,GTUMM,-420,340,452,-420,I,S,0,AA57121294,0,0,220324,NaN,-1,????????,DITON,-155,380,0,6M,"[ZURI, REIM, EGGG]",1,TOM44X,HEGN,EGGD,-1,-1,B38M,0000,GTUMM,-420,380,452,-420,I,S,1297,AA57121294,1,0,220324,NaN,-1,????????,PTGEO,-143,380,0,HH,[REIM],1,1,TOM44X,IS,B38M/M,SDFGHIJ1RWXYZ/LB1,HEGN1700,N0452F340 SOLOX1B SOLOX V603 GIDID DCT KAPIT L...,EGGD0536 EGFF,CODE\n,18,30,18:30,ZURI,TOM44X,True,True,True,NaN,-143,2024-03-22,UTI,False,False,False,False,False,Fal

In [1135]:
jour_1_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_1_utile_traitee)
jour_2_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_2_utile_traitee)
jour_3_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_3_utile_traitee)

jour_1_utile_traitee_sans_doublons = supprimer_doublons(jour_1_utile_traitee_avec_doublons_false)
jour_2_utile_traitee_sans_doublons = supprimer_doublons(jour_2_utile_traitee_avec_doublons_false)
jour_3_utile_traitee_sans_doublons = supprimer_doublons(jour_3_utile_traitee_avec_doublons_false)

C:\Users\latti\AppData\Local\Temp\ipykernel_15568\1227143536.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  1442

delta =  185

delta =  14

delta =  66

delta =  35

delta =  1298

delta =  52

delta =  2

delta =  2

delta =  41

delta =  30

delta =  1438

delta =  198

delta =  256

delta =  538

delta =  77

delta =  1197

delta =  249

delta =  230

delta =  1435

delta =  268

delta =  6

delta =  1444

delta =  16

delta =  16

delta =  41

delta =  41

delta =  175

delta =  565

delta =  48

delta =  6

delta =  22

delta =  230

delta =  29

delta =  0

delta =  113

delta =  154

delta =  152

delta =  115

delta =  21

delta =  426

delta =  134

delta =  8

delta =  38

delta =  33

delta =  38

delta =  102

delta =  55

delta =  116

delta =  104

delta =  3030

delta =  69

delta =  117

delta =  19

delta =  245

delta =  51

delta =  208

delta =  212

delta =  72

delta =  243

delta =  293

delta =  102

delta =  1460

delta =  1

delta =  1447

delta =  80

delta =  147

delta =  123

delta =  57

delta =  215

delta =  61

delta =  14

delta =  749

delta =  

C:\Users\latti\AppData\Local\Temp\ipykernel_15568\1227143536.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  95

delta =  53

delta =  108

delta =  434

delta =  48

delta =  64

delta =  1299

delta =  4

delta =  2

delta =  4

delta =  7

delta =  8

delta =  218

delta =  190

delta =  48

delta =  165

delta =  415

delta =  149

delta =  339

delta =  22

delta =  0

delta =  32

delta =  5

delta =  171

delta =  204

delta =  175

delta =  980

delta =  11

delta =  152

delta =  84

delta =  115

delta =  70

delta =  19

delta =  20

delta =  60

delta =  12

delta =  14

delta =  6

delta =  198

delta =  937

delta =  605

delta =  1460

delta =  1002

delta =  413

delta =  93

delta =  342

delta =  108

delta =  33

delta =  71

delta =  619

delta =  195

delta =  181

delta =  181

delta =  312

delta =  83

delta =  264

delta =  61

delta =  32

delta =  26

delta =  493

delta =  76

delta =  72

delta =  1256

delta =  45

delta =  39

delta =  12

delta =  0

delta =  3

delta =  41

delta =  1

delta =  80

delta =  701

delta =  964

delta =  41

delta =  4


C:\Users\latti\AppData\Local\Temp\ipykernel_15568\1227143536.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False



delta =  81

delta =  12

delta =  22

delta =  16

delta =  153

delta =  300

delta =  157

delta =  15

delta =  190

delta =  30

delta =  55

delta =  509

delta =  1

delta =  1346

delta =  22

delta =  27

delta =  19

delta =  22

delta =  0

delta =  2

delta =  23

delta =  22

delta =  8

delta =  22

delta =  12

delta =  105

delta =  24

delta =  29

delta =  55

delta =  85

delta =  75

delta =  59

delta =  66

delta =  125

delta =  29

delta =  18

delta =  421

delta =  102

delta =  47

delta =  88

delta =  136

delta =  71

delta =  243

delta =  156

delta =  203

delta =  392

delta =  78

delta =  578

delta =  0

delta =  277

delta =  147

delta =  479

delta =  62

delta =  9

delta =  16

delta =  1436

delta =  30

delta =  3

delta =  11

delta =  133

delta =  1474

delta =  75

delta =  98

delta =  40

delta =  87

delta =  42

delta =  82

delta =  29

delta =  38

delta =  35

delta =  15

delta =  74

delta =  1

delta =  204

delta =  386

delta

In [1136]:
jour_2_utile_traitee.head()

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,work1prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,work1final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,numeroPLNMfinal,flightIDfinal,balisefinal,HeurePremiereBalise_final,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,work1realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,case7,case8,case9,case10,case13,case15,case16,case18,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,typePln,heure_de_reference,date_de_reference,utile_inutile,aeronef_de_moins_de_deux_tonnes,vol_a_transmettre,vol_approche,vol_interieur,vol_frontalier,vol_VFR,code_d_exoneration,code_exploitant,compagnie_française,type_d_avion_militaire,PLN_à_verifier_TU,PLN_valide,RAZ_des_invalidites_de_type_TU,invalidite_TU,type_d_indicatif,doublon
0,NSZ4329,ESSA,LFLL,5021,-1,B738,0000,SERPT,-455,380,458,-455,I,S,0,AA57149785,0,FPL,0,230324,NaN,-1,????????,SUTAL,-335,390,0,HE,"[EDDY, REIM, GNEV, LYON]",1,NSZ4329,ESSA,LFLL,5021,-1,B738,0740,SERPT,-455,380,458,-455,I,S,0,AA57149785,1,0,230324,NaN,-1,????????,SUTAL,-331,390,0,XE,"[EDDY, REIM, GNEV, LYON]",1,NSZ4329,ESSA,LFLL,5021,-1,B738,0000,SERPT,-455,250,458,-455,I,S,1132,AA57149785,1,0,230324,4ACA14,-1,NSZ4329,SUTAL,-308,390,0,HE,"[EDDY, REIM, GNEV, LYON]",1,3,NSZ4329,IS,B738/M,SDE2E3GHIJ1LRWXY/LB1,ESSA1625,N0458F380 PETEV N872 ELPAX DCT IBREK DCT ALASA...,LFLL0228 LIMF,CODE/4ACA14 DAT/V DOF/240323 EET/EKDK0040 EDVV...,15,27,15:27,EDDY,NSZ4329LSERPTI,True,True,True,NaN,-308,2024-03-23,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR,False
1,WUK7HS,HEGN,EGKK,6153,-1,A21N,0000,GWUKX,-385,260,479,-385,I,S,0,AA57148092,0,FPL,0,230324,NaN,-1,????????,DITON,-130,360,0,KH,"[ZURI, REIM, EGGG]",1,WUK7HS,HEGN,EGKK,6153,-1,A21N,2763,GWUKX,-385,260,479,-385,I,S,0,AA57148092,0,0,230324,NaN,-1,????????,DITON,-130,360,0,5M,"[ZURI, REIM, EGGG]",1,WUK7HS,HEGN,EGKK,-1,-1,A21N,0000,GWUKX,-385,340,479,-385,I,S,1373,AA57148092,1,0,230324,NaN,-1,????????,PTGEO,-67,340,0,XH,[REIM],1,1,WUK7HS,IS,A21N/M,SDE1E2E3FGIJ1J4RWXY/LB1,HEGN1735,N0479F260 SOLOX1B SOLOX V603 GIDID DCT KAPIT L...,NaN,NaN,18,56,18:56,ZURI,WUK7HS,True,True,True,NaN,-67,2024-03-23,UTI,False,False,False,False,False,False,Z,Z,False,False,False,True,False,[],TR,False
2,TVF7455,ESSA,LFLL,5435,-1,B738,0000,FHTVV,-380,400,461,-380,I,S,0,AA57151002,0,FPL,0,230324,NaN,-1,????????,SUTAL,-262,390,0,HE,"[EDDY, REIM, GNEV, LYON]",1,TVF7455,ESSA,LFLL,5435,-1,B738,0746,FHTVV,-380,400,461,-380,I,S,0,AA57151002,1,0,230324,NaN,-1,????????,SUTAL,-266,390,0,LH,"[EDDY, REIM, GNEV, LYON]",1,TVF7455,ESSA,LFLL,5435,-1,B738,0000,FHTVV,-380,250,461,-380,I,S,1209,AA57151002,1,0,230324,39CEB5,-1,TVF7455,SUTAL,-231,390,0,HE,"[EDDY, REIM, GNEV, LYON]",1,2,TVF7455,IS,B738/M,SDFGHIJ1RWXY/LB1,ESSA1740,N0461F400 PETEV N872 ELPAX DCT IBREK DCT ALASA...,LFLL0220 LFML,CODE/39CEB5 DOF/2

In [1137]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    valeur = int(get_valid_value(row, ['heure_de_reference']))
    heure = (valeur%1440)//60
    min = (valeur%1440)%60
    time_of_departure = f"{(100*heure + min):04}"
    aerodrome_of_departure = get_valid_value(row, ['dep_realise', 'dep_final', 'de_prevu']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['arr_realise', 'arr_final', 'arr_prevu']).ljust(4)
    flight_identification = get_valid_value(row, ['callSign_realise', 'callSign_final', 'callSign_prevu']).ljust(9)
    main_exemption_code = row["code_d_exoneration"].ljust(1)
    aircraft_type = get_valid_value(row, ['typeAvion_realise', 'typeAvion_final', 'typeAvion_prevu']).ljust(7)
    operator = str(row["code_exploitant"]).ljust(3)
    aircraft_registration = str(get_valid_value(row, ['work1realise', 'work1final', 'work1prevu'])).ljust(9)
    comment = ""
    est_off_block_date = get_valid_value(row, ['date_de_reference']).ljust(6)
    ifplid = get_valid_value(row, ['IFPL_realise', 'IFPL_final', 'IFPL_prevu']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = ""
    icao_address = ""
    additional_comment = ""

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment
    }


In [1138]:
jour_123 = pd.concat([jour_1_utile_traitee_avec_doublons_false, jour_2_utile_traitee_avec_doublons_false, jour_3_utile_traitee_avec_doublons_false], ignore_index=True)
jour_123['date_de_reference'] = jour_123['date_de_reference'].dt.strftime('%y%m%d')
jour_123 = jour_123[jour_123["vol_a_transmettre"] == True]
jour_123 = jour_123.apply(format_flight_message, axis=1).tolist()
jour_123 = pd.DataFrame(jour_123)
jour_123['Time of departure/entry'] = pd.to_numeric(jour_123['Time of departure/entry'], errors='coerce')

# Gestion consécutifs

In [1139]:
jour_123.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6722 entries, 0 to 6721
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 6722 non-null   object
 1   Code                            6722 non-null   object
 2   Time of departure/entry         6722 non-null   int64 
 3   Departure aerodrome             6722 non-null   object
 4   Arrival aerodrome               6722 non-null   object
 5   Flight identification           6722 non-null   object
 6   Main Exemption code             6722 non-null   object
 7   Type of aircraft                6722 non-null   object
 8   Operator                        6722 non-null   object
 9   Aircraft Registration           6722 non-null   object
 10  Comment1                        6722 non-null   object
 11  Flight date                     6722 non-null   object
 12  IFPLID                          6722 non-null   

In [1140]:
jour_123.head()

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2
0,0001,F,1154,LFQT,LFQT,0CO458A,Z,DA42,Z,,,240323,,,,,,,,,
1,0003,F,1654,LFMD,EBKT,AAB39R,Z,C25C,Z,OOROK,,240323,AA57161292,,,,,,,,
2,0005,F,800,LFMD,LRBV,AAB98N,Z,C25C,Z,OONSG,,240323,AA57125425,,,,,,,,
3,0011,F,1046,LFPG,KJFK,AAL45,Z,B772,Z,N773A,,240323,AA57148691,,,,,,,,
4,0013,F,1021,LFPG,KDFW,AAL49,Z,B789,Z,N834A,,240323,AA57150706,,,,,,,,


# Donnees ABACUS

In [1141]:
liste_noms_colonnes = ['Origin of message','Flight date','Sequence number','Time of departure/entry','Departure aerodrome','Arrival aerodrome','Type of aircraft','Flight identification','Aircraft Registration','User ICAO Code','User Number','VAT Number','User Nationality','Correction Code','IFPLID','24-bit Aircraft Address','Source of the Aircraft Address','Main Exemption Code','1st Supp. Ex. Code','2nd Supp. Ex. Code','3rd Supp. Ex. Code','Flight Message ID','Claim Number','Pro forma number','Pro forma line number','Bill or Credit Note Reference','MTOW','Weight Coefficient','Distance Coefficient','Service Units','National Route Charge in BZ','Admin. Route Charge in BZ','National Route Charge for SP','Admin. Route Charge for SP','Exemption Indicator','Exemption Code','Flight category','VAT Doc. Ref.','Original VAT Doc. Ref','VAT Rate','VAT Code','VAT on National Route Charge for SP','VAT on Admin. Route Charge for SP']
# ABACUS_Mars = pd.read_csv('../data/ABACUS/LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT', sep=';', usecols = liste_noms_colonnes)

In [1142]:
# ABACUS_Mars_toutes_origines = ABACUS_Mars[ABACUS_Mars["Flight date"] == 240316]
# ABACUS_Mars = ABACUS_Mars[(ABACUS_Mars["Flight date"] == 240316) & (ABACUS_Mars["Origin of message"] == "LF")]

In [1143]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message2(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    time_of_departure = f"{(int(get_valid_value(row, ['Time of departure/entry']))):04}"
    aerodrome_of_departure = get_valid_value(row, ['Departure aerodrome']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['Arrival aerodrome']).ljust(4)
    flight_identification = get_valid_value(row, ['Flight identification']).ljust(9)
    main_exemption_code = row['Main Exemption Code'].ljust(1)
    aircraft_type = get_valid_value(row, ['Type of aircraft']).ljust(7)
    operator = row['Main Exemption Code'].ljust(3)
    aircraft_registration = get_valid_value(row, ['Aircraft Registration']).ljust(9)
    comment = ""
    est_off_block_date = str(get_valid_value(row, ['Flight date'])).ljust(6)
    ifplid = get_valid_value(row, ['IFPLID']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = row['Source of the Aircraft Address']
    icao_address = row['24-bit Aircraft Address']
    additional_comment = ""

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment
    }

In [1144]:
# ABACUS_Mars = ABACUS_Mars.apply(format_flight_message2, axis=1).tolist()
# # Créer un nouveau DataFrame à partir de la liste de dictionnaires
# ABACUS_Mars = pd.DataFrame(ABACUS_Mars)

# ABACUS_Mars['Time of departure/entry'] = pd.to_numeric(ABACUS_Mars['Time of departure/entry'], errors='coerce')

# Fonction de comparaison

In [1145]:
"""## Importation données
Données DTI et CESNAC
"""

widths= [4,1,4,4,4,9,1,7,3,9,21,6,10,4,2,15,15,3,1,6,76]
noms_colonnes = ["Sequence number", "Code", "Time of departure/entry", "Departure aerodrome", "Arrival aerodrome",
                 "Flight identification", "Main Exemption code", "Type of aircraft", "Operator", "Aircraft Registration",
                 "Comment1", "Flight date", "IFPLID", "Planned_aerodrome", "Charging_zone_overflow", "Entry_point", "Exit_point",
                 "Sup_exemption_code", "Source of the Aircraft Address", "24-bit Aircraft Address", "Comment2"]
noms_colonnes_doubles = [f"{x}_{i+1}" for x in noms_colonnes for i in range(2)]

"""# Algorithme de comparaison

## Fonction calcul heures de différences
"""

def difference_time(heure1, heure2, date1, date2, date_fichier1, date_fichier2):
    if heure1 <10:
      h1 = "000" + str(int(heure1))
    elif heure1 < 100:
      h1 = "00" + str(int(heure1))
    elif heure1 < 1000:
      h1 = "0" + str(int(heure1))
    else:
      h1 = str(int(heure1))
    if heure2 <10:
      h2 = "000" + str(int(heure2))
    elif heure2 < 100:
      h2 = "00" + str(int(heure2))
    elif heure2 < 1000:
      h2 = "0" + str(int(heure2))
    else:
      h2 = str(int(heure2))
    try:
      date_obj1 = datetime.strptime(str(int(date1)) + h1, '%y%m%d%H%M')
      date_obj2 = datetime.strptime(str(int(date2)) + h2, '%y%m%d%H%M')
    except ValueError:
        # Si les dates ne sont pas dans le bon format, supposer qu'elles sont les mêmes
        date1 = date_fichier1
        date2 = date_fichier2
        # Convertir les dates en objets datetime
        date_obj1 = datetime.strptime(str(int(date1)) + h1, '%y%m%d%H%M')
        date_obj2 = datetime.strptime(str(int(date2)) + h2, '%y%m%d%H%M')

    # Calculer la différence entre les deux horaires
    difference = date_obj1 - date_obj2

    #Avoir la différence en seconde
    difference_seconde = abs(difference.days * 3600*24 + difference.seconds)
    # Convertir la différence en heures et minutes
    difference_heures = difference_seconde // 3600
    difference_minutes = (difference_seconde % 3600) // 60

    # Retourner la différence sous forme d'entier hhmm
    return difference_heures * 100 + difference_minutes

"""## Définition vol similaire"""

def comparaison_vol(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  arrival = (vol1["Arrival aerodrome"] == vol2["Arrival aerodrome"])
  departure = (vol1["Departure aerodrome"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (arrival and departure and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 45)
  else:
    return False

def comparaison_vol_arrival(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  arrival = (vol1["Arrival aerodrome"] == vol2["Arrival aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (arrival and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 20)
  else:
    return False

def comparaison_vol_departure(vol1, vol2, fichier_date1, fichier_date2) -> bool:
  departure = (vol1["Departure aerodrome"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification"] == vol2[ "Flight identification"])
  if (departure and id) :
    return(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 20)
  else:
    return False

"""## Retrouver vol(s) dans dataset"""

def retrouver_vol(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def retrouver_vol_arrival(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol_arrival(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def retrouver_vol_departure(vol, data,fichier_date1, fichier_date2):
    matching_rows = data[data.apply(lambda row: comparaison_vol_departure(vol, row, fichier_date1, fichier_date2), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

"""## Différences entre 2 vols"""

def differences_vols(vol1, vol2):
  diff = []
  noms = noms_colonnes.copy()
  noms.remove("Sequence number")
  for name in noms:
    if vol1[name] != vol2[name] and not(pd.isna(vol1[name]) and pd.isna(vol2[name])) :
      diff.append(name)
  if len(diff)>0:
    return True
  else:
    return False

"""## Comparaison entre deux datasets d'une journée de vol, format voulu par Eurocontrol"""

def comparaison_data(data1, data2, fichier_date1, fichier_date2):

  # Premières comparaison
  vols_only_data1 = pd.DataFrame(columns=noms_colonnes)
  vols_only_data2 = data2.copy()
  vols_identiques = pd.DataFrame(columns = noms_colonnes)
  index_identiques = 0
  vols_differents = pd.DataFrame(columns=noms_colonnes_doubles)
  for i in range(len(data1)):
    vols, index = retrouver_vol(data1.iloc[i], vols_only_data2, fichier_date1, fichier_date2)
    if len(vols)==0:
      vols_only_data1 = pd.concat([vols_only_data1, data1.iloc[i:i+1]], ignore_index=True)
    else:
      vols_only_data2 = vols_only_data2.drop(index)
      if differences_vols(data1.iloc[i], vols[0]):
        # Convertir les séries en DataFrame
        data1_df = data1.iloc[i].to_frame().T
        vols_df = vols[0].to_frame().T
        # Ajouter un suffixe aux colonnes de la deuxième ligne
        data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
        vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
        # Réinitialiser les index pour aligner les lignes
        data1_df.reset_index(drop=True, inplace=True)
        vols_df.reset_index(drop=True, inplace=True)
        # Créer une nouvelle ligne en alternant les valeurs
        newligne = pd.DataFrame()
        for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
            newligne[colonne1] = data1_df[colonne1]
            newligne[colonne2] = vols_df[colonne2]
        # Réinitialiser l'index pour obtenir une seule ligne
        newligne.reset_index(drop=True, inplace=True)
        vols_differents = pd.concat([vols_differents, newligne], ignore_index=True)
      else:
        vols_identiques.loc[index_identiques] = data1.iloc[i]
        index_identiques+=1
  
  # Premières comparaison
  vols_only_only_data1 = pd.DataFrame(columns=noms_colonnes)
  vols_only_only_data2 = vols_only_data2.copy()
  vols_arr_identique = pd.DataFrame(columns=noms_colonnes_doubles)
  vols_dep_identique = pd.DataFrame(columns=noms_colonnes_doubles)
  for i in range(len(vols_only_data1)):
    vols, index = retrouver_vol_departure(vols_only_data1.iloc[i], vols_only_only_data2, fichier_date1, fichier_date2)
    if len(vols)==0:
      vols, index = retrouver_vol_arrival(vols_only_data1.iloc[i], vols_only_only_data2, fichier_date1, fichier_date2)
      if len(vols)==0:
        vols_only_only_data1 = pd.concat([vols_only_only_data1, vols_only_data1.iloc[i:i+1]], ignore_index=True)
      else:
        vols_only_only_data2 = vols_only_only_data2.drop(index)
        # Convertir les séries en DataFrame
        data1_df = vols_only_data1.iloc[i].to_frame().T
        vols_df = vols[0].to_frame().T
        # Ajouter un suffixe aux colonnes de la deuxième ligne
        data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
        vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
        # Réinitialiser les index pour aligner les lignes
        data1_df.reset_index(drop=True, inplace=True)
        vols_df.reset_index(drop=True, inplace=True)
        # Créer une nouvelle ligne en alternant les valeurs
        newligne = pd.DataFrame()
        for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
            newligne[colonne1] = data1_df[colonne1]
            newligne[colonne2] = vols_df[colonne2]
        # Réinitialiser l'index pour obtenir une seule ligne
        newligne.reset_index(drop=True, inplace=True)
        vols_arr_identique = pd.concat([vols_arr_identique, newligne], ignore_index=True)
    else:
      vols_only_only_data2 = vols_only_only_data2.drop(index)
      # Convertir les séries en DataFrame
      data1_df = vols_only_data1.iloc[i].to_frame().T
      vols_df = vols[0].to_frame().T
      # Ajouter un suffixe aux colonnes de la deuxième ligne
      data1_df.columns = [colonne + '_1' for colonne in data1_df.columns]
      vols_df.columns = [colonne + '_2' for colonne in vols_df.columns]
      # Réinitialiser les index pour aligner les lignes
      data1_df.reset_index(drop=True, inplace=True)
      vols_df.reset_index(drop=True, inplace=True)
      # Créer une nouvelle ligne en alternant les valeurs
      newligne = pd.DataFrame()
      for colonne1, colonne2 in zip(data1_df.columns, vols_df.columns):
          newligne[colonne1] = data1_df[colonne1]
          newligne[colonne2] = vols_df[colonne2]
      # Réinitialiser l'index pour obtenir une seule ligne
      newligne.reset_index(drop=True, inplace=True)
      vols_dep_identique = pd.concat([vols_dep_identique, newligne], ignore_index=True)

  # Traitement final
  vols_differents.insert(0, 'compare', "vols_identiques")
  vols_identiques.insert(0, 'compare', "vols_quasi_identiques")
  vols_only_only_data1.insert(0, 'compare', "vols_only_data_1")
  vols_only_only_data2.insert(0, 'compare', "vols_only_data_2")
  vols_dep_identique.insert(0, 'compare', "vols_arr_diff")
  vols_arr_identique.insert(0, 'compare', "vols_dep_diff")

  colonnes_dupliquees = [vols_identiques.iloc[:, 0]] + [vols_identiques[colonne].rename(f"{colonne}_{i+1}") for colonne in vols_identiques.columns[1:] for i in range(2)]
  vols_identiques = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data1.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data1.iloc[:, 0])
  for colonne in vols_only_only_data1.columns[1:]:
      colonnes_dupliquees.append(vols_only_only_data1[colonne].rename(f"{colonne}_1"))
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data1), name=f"{colonne}_2"))
  vols_only_only_data1 = pd.concat(colonnes_dupliquees, axis=1)

  vols_only_only_data2.reset_index(drop=True, inplace=True)
  colonnes_dupliquees = []
  colonnes_dupliquees.append(vols_only_only_data2.iloc[:, 0])
  for colonne in vols_only_only_data2.columns[1:]:
      colonnes_dupliquees.append(pd.Series([np.nan] * len(vols_only_only_data2), name=f"{colonne}_1"))
      colonnes_dupliquees.append(vols_only_only_data2[colonne].rename(f"{colonne}_2"))
  vols_only_only_data2 = pd.concat(colonnes_dupliquees, axis=1)

  result = pd.concat([vols_differents, vols_identiques, vols_only_only_data1, vols_only_only_data2, vols_dep_identique, vols_arr_identique])
  result = result.drop("Sequence number_1", axis=1)
  result = result.drop("Sequence number_2", axis=1)
  result = result.reset_index(drop=True)
  return(result)

# Result

In [1146]:
# jour_cible = jour_123[(jour_123["Flight date"] == "240316") | (jour_123["Flight date"] == "      ")]

In [1147]:
target_date_str =dateAnalyse
target_date = datetime.strptime(target_date_str, "%d-%m-%Y")
target_date_str = target_date.strftime("%y%m%d")
ABACUS_Mars = pd.read_csv('../data/ABACUS/LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT', sep=';', usecols = liste_noms_colonnes)
ABACUS_Mars_toutes_origines = ABACUS_Mars[ABACUS_Mars["Flight date"] == int(target_date_str)]

ABACUS_Mars = ABACUS_Mars[(ABACUS_Mars["Flight date"] == int(target_date_str)) & (ABACUS_Mars["Origin of message"] == "LF")]
ABACUS_Mars = ABACUS_Mars.apply(format_flight_message2, axis=1).tolist()
# Créer un nouveau DataFrame à partir de la liste de dictionnaires
ABACUS_Mars = pd.DataFrame(ABACUS_Mars)

ABACUS_Mars['Time of departure/entry'] = pd.to_numeric(ABACUS_Mars['Time of departure/entry'], errors='coerce')
# Filter jour_123 for the target date and blank dates
jour_cible = jour_123[(jour_123["Flight date"] == target_date_str) | (jour_123["Flight date"] == "      ")]
    

C:\Users\latti\AppData\Local\Temp\ipykernel_15568\3279519034.py:4: DtypeWarning: Columns (18,35) have mixed types. Specify dtype option on import or set low_memory=False.
  ABACUS_Mars = pd.read_csv('../data/ABACUS/LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT', sep=';', usecols = liste_noms_colonnes)


In [1148]:
jour_123['Flight date'].value_counts()

Flight date
240325    2599
240324    2136
240323    1969
240322      18
Name: count, dtype: int64

In [1149]:
jour_cible.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2136 entries, 1973 to 6439
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 2136 non-null   object
 1   Code                            2136 non-null   object
 2   Time of departure/entry         2136 non-null   int64 
 3   Departure aerodrome             2136 non-null   object
 4   Arrival aerodrome               2136 non-null   object
 5   Flight identification           2136 non-null   object
 6   Main Exemption code             2136 non-null   object
 7   Type of aircraft                2136 non-null   object
 8   Operator                        2136 non-null   object
 9   Aircraft Registration           2136 non-null   object
 10  Comment1                        2136 non-null   object
 11  Flight date                     2136 non-null   object
 12  IFPLID                          2136 non-null   ob

In [1150]:
dataresult = comparaison_data(jour_cible, ABACUS_Mars, int(target_date_str), int(target_date_str))

In [1151]:
dataresult["compare"].value_counts()

compare
vols_identiques     2095
vols_only_data_1      35
vols_only_data_2       7
vols_arr_diff          6
Name: count, dtype: int64

In [1152]:
dataresult[dataresult["compare"] == "vols_only_data_2"]

,compare,Code_1,Code_2,Time of departure/entry_1,Time of departure/entry_2,Departure aerodrome_1,Departure aerodrome_2,Arrival aerodrome_1,Arrival aerodrome_2,Flight identification_1,Flight identification_2,Main Exemption code_1,Main Exemption code_2,Type of aircraft_1,Type of aircraft_2,Operator_1,Operator_2,Aircraft Registration_1,Aircraft Registration_2,Comment1_1,Comment1_2,Flight date_1,Flight date_2,IFPLID_1,IFPLID_2,Planned_aerodrome_1,Planned_aerodrome_2,Charging_zone_overflow_1,Charging_zone_overflow_2,Entry_point_1,Entry_point_2,Exit_point_1,Exit_point_2,Sup_exemption_code_1,Sup_exemption_code_2,Source of the Aircraft Address_1,Source of the Aircraft Address_2,24-bit Aircraft Address_1,24-bit Aircraft Address_2,Comment2_1,Comment2_2
2130,vols_only_data_2,NaN,F,NaN,1944,NaN,LFPM,NaN,LFPT,NaN,N827JC,NaN,Z,NaN,P32R,NaN,Z,NaN,N827JC,NaN,,NaN,240324,NaN,AA57184848,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,AB4BA7,NaN,
2131,vols_only_data_2,NaN,F,NaN,4,NaN,LFPG,NaN,SCEL,NaN,AFR406,NaN,Z,NaN,B772,NaN,Z,NaN,FGSPL,NaN,,NaN,240324,NaN,AA57163334,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,3949F8,NaN,
2132,vols_only_data_2,NaN,F,NaN,224,NaN,FKYS,NaN,EBBR,NaN,BEL3DY,NaN,Z,NaN,A333,NaN,Z,NaN,OOSFX,NaN,,NaN,240324,NaN,AA57159260,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,44CCD8,NaN,
2133,vols_only_data_2,NaN,F,NaN,2341,NaN,LFPG,NaN,VIDP,NaN,FDX5384,NaN,Z,NaN,B77L,NaN,Z,NaN,N870FD,NaN,,NaN,240324,NaN,AA57163518,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,ABF6D7,NaN,
2134,vols_only_data_2,NaN,F,NaN,14,NaN,LFMT,NaN,LFPG,NaN,AIA172,NaN,Z,NaN,E145,NaN,Z,NaN,FHGYM,NaN,,NaN,240324,NaN,AA57185481,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,399B0C,NaN,
2135,vols_only_data_2,NaN,F,NaN,2014,NaN,DTTJ,NaN,LFPO,NaN,TVF8603,NaN,Z,NaN,B738,NaN,Z,NaN,FHUYK,NaN,,NaN,240324,NaN,AA57176583,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,1.0,NaN,39D30A,NaN,
2136,vols_only_data_2,NaN,F,NaN,1710,NaN,LFQQ,NaN,LFQQ,NaN,FGTRY,NaN,Z,NaN,C525,NaN,Z,NaN,FGTRY,NaN,,NaN,240324,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,


In [1153]:
ABACUS_Mars[ABACUS_Mars["Flight identification"] == "FGZBI    "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [1154]:
jour_cible[jour_cible["Flight identification"] == "FGZBI    "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [1155]:
ABACUS_Mars[ABACUS_Mars["Flight identification"] == "FGNTS    "]


,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [1156]:
jour_cible[jour_cible["Flight identification"] == "FGNTS    "]

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2


In [1157]:
jour_2[jour_2["callSign_prevu"] == "CTM2049"]

,callSign_prevu,dep_prevu,arr_prevu,numCautra_prevu,dateRelative_prevu,typeAvion_prevu,work_prevu,work1prevu,heuresDep_prevu,RFL_prevu,vitesse_prevu,EOBT_prevu,regleVol_prevu,typeVol_prevu,HeurePremiereBaliseActive_prevu,IFPL_prevu,plnActive_prevu,typePlnStan,plnAnnule_prevu,dateBlock_prevu,adresseModeS_prevu,numeroPLNMprevu,flightIDprevu,baliseprevu,HeurePremiereBalise_prevu,listeBalisesprevu,indicateurprevu,carteprevu,centreTraverséprevu,listeRangPremierprevu,callSign_final,dep_final,arr_final,numCautra_final,dateRelative_final,typeAvion_final,work_final,work1final,heuresDep_final,RFL_final,vitesse_final,EOBT_final,regleVol_final,typeVol_final,HeurePremiereBaliseActive_final,IFPL_final,plnActive_final,plnAnnule_final,dateBlock_final,adresseModeS_final,numeroPLNMfinal,flightIDfinal,balisefinal,HeurePremiereBalise_final,listeBalisesfinal,indicateurfinal,cartefinal,centreTraverséfinal,listeRangPremierfinal,callSign_realise,dep_realise,arr_realise,numCautra_realise,dateRelative_realise,typeAvion_realise,work_realise,work1realise,heuresDep_realise,RFL_realise,vitesse_realise,EOBT_realise,regleVol_realise,typeVol_realise,HeurePremiereBaliseActive_realise,IFPL_realise,plnActive_realise,plnAnnule_realise,dateBlock_realise,adresseModeS_realise,numeroPLNMrealise,flightIDrealise,baliserealise,HeurePremiereBalise_realise,listeBalisesrealise,indicateurrealise,carterealise,centreTraversérealise,listeRangPremierrealise,rangTransactiontransaction,case7,case8,case9,case10,case13,case15,case16,case18,heure,minute,accuseTrttransaction,ccrArrival,finaltransaction,isPrevu,isRealise,isFinal,typePln,heure_de_reference,date_de_reference,utile_inutile


In [1158]:
def comparaison_vol_all(vol1, vol2) -> bool:
  arrival = (vol1["Arrival aerodrome_1"] == vol2["Arrival aerodrome"])
  departure = (vol1["Departure aerodrome_1"] == vol2["Departure aerodrome"])
  id = (vol1["Flight identification_1"].rstrip() == vol2["Flight identification"])
  return (arrival and departure and id)

  #(difference_time(vol1["Time of departure/entry"], vol2["Time of departure/entry"], vol1["Flight date"], vol2["Flight date"], fichier_date1, fichier_date2) < 130)
  
def retrouver_vol_all(vol, data):
    matching_rows = data[data.apply(lambda row: comparaison_vol_all(vol, row), axis=1)]
    index = matching_rows.index
    return ([matching_rows.iloc[0]], index[0]) if len(matching_rows) >0 else ([],0)

def analyse(data_en_plus):
    data_en_plus["Raison"] = ""
    for i in range(len(data_en_plus)):
        vols_abacus_all = retrouver_vol_all(data_en_plus.iloc[i], ABACUS_Mars_toutes_origines)
        if len(vols_abacus_all[0]) > 0:
            if vols_abacus_all[0][0]["Origin of message"] != "LF":
                data_en_plus["Raison"].iloc[i] = "Origin_of_message"
            elif abs(vols_abacus_all[0][0]["Time of departure/entry"] - data_en_plus.iloc[i]["Time of departure/entry_1"]) > 100:
                data_en_plus["Raison"].iloc[i] = "Time"
    return data_en_plus

In [1159]:
data_en_plus = dataresult[dataresult["compare"] == "vols_only_data_1"]

In [1160]:
data_en_plus = analyse(data_en_plus)

C:\Users\latti\AppData\Local\Temp\ipykernel_15568\4155042169.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en_plus["Raison"] = ""
C:\Users\latti\AppData\Local\Temp\ipykernel_15568\4155042169.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en_plus["Raison"].iloc[i] = "Time"
C:\Users\latti\AppData\Local\Temp\ipykernel_15568\4155042169.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [1161]:
data_en_plus

,compare,Code_1,Code_2,Time of departure/entry_1,Time of departure/entry_2,Departure aerodrome_1,Departure aerodrome_2,Arrival aerodrome_1,Arrival aerodrome_2,Flight identification_1,Flight identification_2,Main Exemption code_1,Main Exemption code_2,Type of aircraft_1,Type of aircraft_2,Operator_1,Operator_2,Aircraft Registration_1,Aircraft Registration_2,Comment1_1,Comment1_2,Flight date_1,Flight date_2,IFPLID_1,IFPLID_2,Planned_aerodrome_1,Planned_aerodrome_2,Charging_zone_overflow_1,Charging_zone_overflow_2,Entry_point_1,Entry_point_2,Exit_point_1,Exit_point_2,Sup_exemption_code_1,Sup_exemption_code_2,Source of the Aircraft Address_1,Source of the Aircraft Address_2,24-bit Aircraft Address_1,24-bit Aircraft Address_2,Comment2_1,Comment2_2,Raison
2095,vols_only_data_1,F,NaN,3,NaN,LFPB,NaN,LFBO,NaN,ASJ01TJ,NaN,Z,NaN,C510,NaN,Z,NaN,FGMTJ,NaN,,NaN,240324,NaN,AA57195229,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2096,vols_only_data_1,F,NaN,1615,NaN,LFML,NaN,LFKC,NaN,CCM38QO,NaN,Z,NaN,AT76,NaN,Z,NaN,FHXKE,NaN,,NaN,240324,NaN,AA57162870,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2097,vols_only_data_1,F,NaN,855,NaN,LFML,NaN,LFKJ,NaN,CCM46TY,NaN,Z,NaN,AT76,NaN,Z,NaN,FHXKY,NaN,,NaN,240324,NaN,AA57162515,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2098,vols_only_data_1,F,NaN,18,NaN,LFPO,NaN,SOCA,NaN,CTM001,NaN,M,NaN,A332,NaN,003,NaN,FRARF,NaN,,NaN,240324,NaN,AA57189337,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2099,vols_only_data_1,F,NaN,1623,NaN,LFMN,NaN,EGGW,NaN,EDC959,NaN,Z,NaN,E550,NaN,Z,NaN,GWLKR,NaN,,NaN,240324,NaN,AA57157713,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2100,vols_only_data_1,F,NaN,600,NaN,LFMN,NaN,LFBO,NaN,EJU437U,NaN,Z,NaN,A320,NaN,Z,NaN,OEICK,NaN,,NaN,240324,NaN,AA57160360,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2101,vols_only_data_1,F,NaN,1447,NaN,LFMN,NaN,LFQQ,NaN,EJU675Z,NaN,Z,NaN,A320,NaN,Z,NaN,OEICK,NaN,,NaN,240324,NaN,AA57184743,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2102,vols_only_data_1,F,NaN,2055,NaN,LFMN,NaN,LFSB,NaN,EZS597T,NaN,Z,NaN,A320,NaN,Z,NaN,HBJXL,NaN,,NaN,240324,NaN,AA57177196,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2103,vols_only_data_1,F,NaN,1522,NaN,LFSL,NaN,LFRI,NaN,FHLRA,NaN,Z,NaN,DA62,NaN,Z,NaN,FHLRA,NaN,,NaN,240324,NaN,AA57103163,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,
2104,vols_only_data_1,F,NaN,1055,NaN,LFPN,NaN,LFLB,NaN,FHSKJ,NaN,Z,NaN,VTOR,NaN,Z,NaN,FHSKJ,NaN,,NaN,240324,NaN,AA57177253,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,,NaN,


In [1162]:
data_en_plus[['compare','Time of departure/entry_1','Raison']]

,compare,Time of departure/entry_1,Raison
2095,vols_only_data_1,3,
2096,vols_only_data_1,1615,
2097,vols_only_data_1,855,
2098,vols_only_data_1,18,
2099,vols_only_data_1,1623,
2100,vols_only_data_1,600,
2101,vols_only_data_1,1447,
2102,vols_only_data_1,2055,
2103,vols_only_data_1,1522,
2104,vols_only_data_1,1055,


In [1163]:
colonnes_a_afficher = ['compare', 'Time of departure/entry_1', 'Departure aerodrome_1', 'Arrival aerodrome_1', 'Flight identification_1', 'Type of aircraft_1', 'Raison']

data_en_plus[colonnes_a_afficher]

,compare,Time of departure/entry_1,Departure aerodrome_1,Arrival aerodrome_1,Flight identification_1,Type of aircraft_1,Raison
2095,vols_only_data_1,3,LFPB,LFBO,ASJ01TJ,C510,
2096,vols_only_data_1,1615,LFML,LFKC,CCM38QO,AT76,
2097,vols_only_data_1,855,LFML,LFKJ,CCM46TY,AT76,
2098,vols_only_data_1,18,LFPO,SOCA,CTM001,A332,
2099,vols_only_data_1,1623,LFMN,EGGW,EDC959,E550,
2100,vols_only_data_1,600,LFMN,LFBO,EJU437U,A320,
2101,vols_only_data_1,1447,LFMN,LFQQ,EJU675Z,A320,
2102,vols_only_data_1,2055,LFMN,LFSB,EZS597T,A320,
2103,vols_only_data_1,1522,LFSL,LFRI,FHLRA,DA62,
2104,vols_only_data_1,1055,LFPN,LFLB,FHSKJ,VTOR,
